In [201]:
from sqlalchemy import text, create_engine
import pandas as pd
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
import numpy as np 
import fritz

In [ ]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [ ]:
consultant_name_df = pd.read_excel('G:/Post Construction/metadata csvs/consultants.xlsx', encoding = 'utf-8-sig')
consultant_name_lst = consultant_name_df["consultant_name"].tolist()

In [ ]:
with engine.connect() as conn: ## with context manager with, we don't have to close the connection everytime we run queries
    query1 = "SELECT * FROM pdfs;"
    df1 = pd.read_sql(query1, conn)

In [ ]:
main_consultant_name = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\t1nipun\AppData\Local\Tesseract-OCR\tesseract.exe'
for file in df1['pdfName']:
    pages = convert_from_path("G:/Post Construction/PCMR_All/" + file + '.pdf', 500, last_page = 1)
    for page in pages:
        filename = file +'_Page1.jpg'
        page.save("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename, 'JPEG')
        text = str(((pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename))))).lower()
        consultant_name = []
        for name in consultant_name_lst:
            if name.lower() in text:
                consultant_name.append(name)
        main_consultant_name.append(consultant_name)

In [ ]:
df1['consultant_name'] = main_consultant_name
mapping_dict = df1.groupby('filingId')['consultant_name'].apply(lambda x: x.values.tolist()).to_dict()

In [ ]:
def flatten(lst_of_lsts):
    output = []
    for element in lst_of_lsts:
        if type(element) == list:
            output.extend(element)
    return list(set(output))

In [200]:
newdict = {key: flatten(value) for key, value in mapping_dict.items()}
newdict[775971] = ['Golder Associates']
newdict[786189] = ['TERA Environmental Consultants']
df1['consultant_name'] = df1['filingId'].map(newdict)

In [ ]:
def clean(mystr):
#     my_new_str = re.sub("(\\W| +)"," ", mystr) #remove anything that is not a letter or number
#     my_new_str = re.sub(r'\s+', ' ', my_new_str) #eliminate duplicate whitespaces
#     my_new_str = re.sub(r"\b\d+\b", "", my_new_str)
    my_new_str = re.sub(r"[^a-zA-Z0-9-&]+",' ', mystr) #remove special characters
#     my_new_str = my_new_str.strip()
    return my_new_str

In [ ]:
texts = {}
consultants_all = []

source_text_dir =  os.listdir('G:/Post Construction/PDF_Final_text')
source_pdf_dir = "G:/Post Construction/PCMR_All/"
for file in source_text_dir:
    with open("G:/Post Construction/PDF_Final_text/" + file,'r', encoding = 'utf-8-sig') as corpus:
        text_files.append(file.replace(".txt",""))
        doc = fitz.open(os.path.join(source_pdf_dir,file.replace('.txt','.pdf')))
        names_pdf.append(file.replace('.txt','.pdf'))
        page_content_str = ''
        text = corpus.read().lower()
        text = clean(text)
        texts_before_val[file.replace(".txt","")] = text
        if references_count > 3:
            before_third = text.split("references")
            result_before_third = '_'.join(before_third[:3]), '_'.join(before_third[3:])
            texts[file] = result_before_third[0]
        elif references_count == 2 or 3:
            before_second = text.split("references") #reference: https://stackoverflow.com/questions/47927039/reading-a-file-until-a-specific-character-in-python
            result_before_second = '_'.join(before_second[:2]), '_'.join(before_second[2:])  # reference: https://stackoverflow.com/questions/17060039/split-string-at-nth-occurrence-of-a-given-character
            texts[file] = result_before_second[0]
        else:
            texts[file] = text

In [199]:
columns  = ['pdfId', 'pdfName', 'filingId', 'totalPages', 'application_id', 'submitter', 'company', 'consultant_name']
df1.to_csv('validation1.csv', encoding = 'utf-8-sig', columns = columns)

In [203]:
df1.head(50)

,pdfId,pdfName,pdfSize,filingId,date,totalPages,status,xmlContent,application_id,submitter,company,consultant_name
0,337444,A0L2F6 - 002 Ekwan Post Construction Monitorin...,0.05,337440,2004-10-06 14:46:30,11,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",268876,EnCana Ekwan Pipeline Inc.,EnCana Ekwan Pipeline Inc.,[AXYS Environmental Consulting Ltd]
1,340413,A0L3K4 - Kwoen Facilities and Re-injection Ext...,1.02,340412,2004-11-15 11:07:40,19,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",139307,"Westcoast Energy Inc., a Duke Energy company",Westcoast Energy Inc.,[]
2,399963,A0T5H7 - Kwoen Re-injection Pipeline - 3rd Yea...,1.60,399962,2006-03-06 16:18:03,25,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",139307,"Westcoast Energy Inc., carrying on business as...",Westcoast Energy Inc.,[]
3,408901,A0U3V0 - 002 Post Construction Monitoring - Au...,0.54,409162,2006-05-30 09:23:58,29,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",268876,Alan K. Haskey,EnCana Ekwan Pipeline Inc.,[AXYS Environmental Consulting Ltd]
4,476639,B-14R Appendix MPLA-SAPL IR 43 b) - TERA Post ...,7.16,476698,2007-09-10 12:30:32,301,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",465178,Enbridge Pipelines Inc.,Enbridge Pipelines Inc.,[]
5,485515,A1C1E9 - 2007 PCM Report Year 1,2.00,485763,2007-11-16 15:21:43,22,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",438887,Husky Oil Operations Limited,Husky Oil Operations Limited,[TERA Environmental Consultants]
6,485680,A1C1E6 - 2007 PCM Report Year 1,2.63,485676,2007-11-16 14:36:23,15,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",401233,ConocoPhillips Canada,Burlington Resources Canada (Hunter) Ltd.,[TERA Environmental Consultants]
7,495406,A1D2H0 - Stittsville Post-Construction Monitor...,2.82,495396,2008-01-29 09:17:53,16,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",379850,TransCanada PipeLines Limited,TransCanada PipeLines Limited,[]
8,538748,A1H6G5 - Post-Construction Monitoring Report Y...,3.15,538744,2008-11-04 15:19:56,12,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",485442,ConocoPhillips Canada,Burlington Resources Canada (Hunter) Ltd.,[TERA Environmental Consultants]
9,538758,A1H6G8 - Post-Construction Monitoring Report Y...,1.78,538754,2008-11-04 15:30:11,17,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",401233,ConocoPhillips Canada,Burlington Resources Canada (Hunter) Ltd.,[TERA Environmental Consultants]
